Section 1 : PE Analysis

In [2]:
import pandas as pd
import os

In [3]:
file_path = r"C:\Users\surji\Desktop\Quant_Poject\Perfomance_metrics\data\pe_ratio_data.csv"
df = pd.read_csv(file_path, parse_dates=['Date'])

df = df.sort_values(['TickerName', 'Date']).reset_index(drop=True)
tickers = df['TickerName'].unique()

These are the rolling windows for test 

In [4]:
rolling_windows = [20, 60, 120, 180, 250, 500]

In [5]:

experiment_results = []

for window in rolling_windows:
    print(f"Processing rolling window: {window}")
    df_window = df.copy()

    df_window['Rolling_P10'] = df_window.groupby('TickerName')['PE'].transform(
        lambda x: x.rolling(window=window,min_periods=5).quantile(0.10)
    )
    df_window['Rolling_P90'] = df_window.groupby('TickerName')['PE'].transform(
        lambda x: x.rolling(window=window, min_periods=5).quantile(0.90)
    )

    df_window['Signal'] = 0
    df_window.loc[df_window['PE'] < df_window['Rolling_P10'], 'Signal'] = 1
    df_window.loc[df_window['PE'] > df_window['Rolling_P90'], 'Signal'] = -1
    df_window['RollingWindow'] = window

    experiment_results.append(df_window)
df_experiments = pd.concat(experiment_results, ignore_index=True)


Processing rolling window: 20
Processing rolling window: 60
Processing rolling window: 120
Processing rolling window: 180
Processing rolling window: 250
Processing rolling window: 500


In [5]:
df.groupby('TickerName')['Date'].min()
df.groupby('TickerName').size()


TickerName
AAPL     63
GOOGL    64
META     50
MSFT     63
TSLA     59
dtype: int64

This is just and experiment code to check what values are returning for the given window before saving it in excel

In [10]:
df_experiments[df_experiments['Rolling_P10'].isna() & (df_experiments['Date'] < '2015')]


,Date,TickerName,StockPrice,NetEPS,PE,Rolling_P10,Rolling_P90,Signal,RollingWindow
0,2009-12-31,AAPL,6.33,$0.31,20.48,NaN,NaN,0,20
1,2010-03-31,AAPL,7.06,$0.36,19.45,NaN,NaN,0,20
2,2010-06-30,AAPL,7.56,$0.44,17.17,NaN,NaN,0,20
3,2010-09-30,AAPL,8.53,$0.54,15.76,NaN,NaN,0,20
63,2009-12-31,GOOGL,15.42,$0.51,30.19,NaN,NaN,0,20
...,...,...,...,...,...,...,...,...,...
1675,2010-09-30,MSFT,18.62,$2.32,8.02,NaN,NaN,0,500
1735,2011-03-31,TSLA,1.85,$-0.43,0.00,NaN,NaN,0,500
1736,2011-06-30,TSLA,1.94,$-0.14,0.00,NaN,NaN,0,500
1737,2011-09-30,TSLA,1.63,$-0.15,0.00,NaN,NaN,0,500


In [11]:
file_path = r"C:\Users\surji\Desktop\Quant_Poject\Perfomance_metrics\data\pe_ratio_rolling_analysis.csv"
df_experiments.to_csv(file_path, index=False)
print(" Data saved ")



 Data saved 


In [12]:
folder_path = r"C:\Users\surji\Desktop\Quant_Poject\Perfomance_metrics\data"
file_name = "experiment_tracking.csv"
full_path = os.path.join(folder_path, file_name)

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

experiment_tracking = pd.DataFrame(columns=[
    'TickerName', 'RollingWindow', 'ROI', 'MaxDrawdown', 'SharpeRatio', 'ROI_to_MDD'
])
experiment_tracking.to_csv(full_path, index=False)
print("Experiment tracking file has been created")


Experiment tracking file has been created
